# 🏔️ Mount Etna Sentinel-1 Displacement Mapping with EDK-SAR

This example demonstrates how to use **EDK-SAR** to generate a line-of-sight (LOS) displacement map and coherence map from Sentinel-1 SLC data over Mount Etna.

### Workflow Overview
1. Initialize the EDK-SAR environment  
2. Co-register Sentinel-1 SLCs  
3. Generate interferograms  
4. Load and geocode interferogram and coherence files  
5. Compute phase and LOS displacement  
6. Export results as GeoTIFFs  
7. Visualize outputs on an interactive folium map  

This notebook demonstrates an **end-to-end InSAR analysis** with minimal code.


In [ ]:
import edk_sar as es
import xarray as xr
import numpy as np
import webbrowser
import importlib
from edk_sar.workflows.base import xarray_accessor  
from edk_sar.workflows.base.constants import SENTINEL_WAVELENGTH as WAVELENGTH
import folium

# Reload the module to get the latest changes
importlib.reload(xarray_accessor)

In [ ]:
# Initialize EDK-SAR environment
env_path = ".env"
es.init(env_path)
print("[OK] Environment initialized")


In [ ]:
# Define path to Mount Etna Sentinel-1 SLCs
slc_path = "edk_sar/data/slc"

# Run preprocessing workflows
es.workflows.coregister.run(slc_path)
es.workflows.interferograms.run(slc_path)
print("[OK] Coregistration and interferogram generation complete")


In [ ]:
interferograms = xr.open_dataarray(
    "path to filt_fine.int.vrt",
    engine="rasterio"
)
coherence = xr.open_dataarray(
    "path to filt_fine.cor.vrt",
    engine="rasterio"
)

In [ ]:
geocoded_interferograms = interferograms.edk.geocode()
geocoded_coherence = coherence.edk.geocode()

In [ ]:
# Compute phase from complex interferogram
phase = xr.apply_ufunc(np.angle, geocoded_interferograms)
geocoded_phase = phase.edk.geocode()

# Compute LOS displacement
displacement = (geocoded_phase * WAVELENGTH) / (4 * np.pi)
displacement.attrs.update({
    "units": "meters",
    "description": "Line-of-sight displacement over Mount Etna"
})
print("[OK] LOS displacement computed")


In [ ]:
print("Geocoded Interferogram:")
print(geocoded_interferograms)

print("\nPhase:")
print(phase)

print("\nGeocoded Coherence:")
print(geocoded_coherence)

print("\nPhase again:")
print(phase)


In [ ]:
# Export all outputs as GeoTIFFs
geocoded_phase.edk.export("examples/mount_etna_phase.tif")
geocoded_coherence.edk.export("examples/mount_etna_coherence.tif")
displacement.edk.export("examples/mount_etna_displacement.tif")
print("[OK] Exported GeoTIFFs to examples/")


In [ ]:
geocoded_phase.edk.plot(colors=['purple','yellow','red'], opacity=0.7)

In [ ]:
geocoded_coherence.edk.plot(colors=['blue','cyan','green'], opacity=0.7)

In [ ]:
displacement.edk.plot(colors=['blue','cyan','green'], opacity=0.7)